In [10]:
import math 
from pathlib import Path 
from types import SimpleNamespace
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as Adam 
from torch.utils.data import dataloader 
from utilities import get_dataloaders
import wandb

ModuleNotFoundError: No module named 'utilities'

In [5]:
INPUT_SIZE = 3 * 16 * 16
OUTPUT_SIZE = 5 
HIDDEN_SIZE = 256 
NUM_WORKERS = 2 
CLASSES = ["hero", "non-hero", "food", "spell", "side-facing"]
DATA_DIR = Path("./data/")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
def get_model(dropout):
    "Simple MLP with Dropout"
    return nn.Sequential(
        nn.Flatten(),
        nn.Linear(INPUT_SIZE, HIDDEN_SIZE),
        nn.BatchNorm1d(HIDDEN_SIZE),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE)
    ).to(DEVICE)

In [7]:
# Define config object to store hyperparameters
config = SimpleNamespace(   
    epochs=2,
    batch_size=128,
    lr = 1e-5,
    dropout = 0.5,
    slice_size = 10_000,
    valid_pct = 0.2,
)

In [11]:
def train_model(config):
    "Train a model with a given config"

    wandb.init(project = "eval_crypo_performance", config = config)

    # Get the data 
    train_dl, valid_dl = dataloader(DATA_DIR, config)

    n_steps_per_epoch = math.ceil(len(train_dl.dataset) / train_dl.batch_size)

    model = get_model(config)

    loss_func = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr = config.lr)

    example_ct = 0 

    for epoch in tqdm(range(config.epochs), total=config.epochs): 
        model.train()

        for step, (images, labels) in enumerate(train_dl):
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            outputs = model(images)
            train_loss = loss_func(outputs, labels)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            example_ct += len(images)
            metrics = {
                "train/train_loss": train_loss,
                "train/epoch": epoch + 1,
                "train/example_ct": example_ct
            }
            wandb.log(metrics)

        # compute validation metrics
        val_loss, accuracy = validate_model(model, valid_dl, loss_func)
        val_metrics = {
            "val/val_loss": val_loss,
            "val/accuracy": accuracy
        }
        wandb.log(val_metrics)
    
    wandb.finish() 

In [15]:
def validate_model(model, valid_dl, loss_func):
    "Compute the performance of the model on the validation set"
    model.eval()
    val_loss = 0.0
    correct = 0 

    with torch.inference_mode():
        for i, (images, labels) in enumerate(valid_dl):
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            outputs = model(images)
            val_loss += loss_func(outputs, labels) * labels.size(0)

            # compute accuracy and accumulate 

            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

    return val_loss / len(valid_dl.dataset), correct / len(valid_dl.dataset)

In [13]:
wandb.login(anonymous="allow")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: leoncen0-iga. Use `wandb login --relogin` to force relogin


True

In [14]:
train_model(config)

TypeError: 'module' object is not callable

In [ ]:
config.lr = 1e-4
train_model(config)

In [ ]:
config.dropout = 0.1 
config.epochs = 1 
train_model(config)

In [ ]:
config.lr = 1e-3
train_model(config)